In [ ]:
'''
Mount Google Drive, copy data to runtime, and unzip folders

Make sure to put a link to "EC 523 Project" in your main google drive!
'''

from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/'Deep Learning Proj'/train.zip /content
! cp /content/drive/MyDrive/'Deep Learning Proj'/test.zip /content
! cp /content/drive/MyDrive/'Deep Learning Proj'/val.zip /content
! cp /content/drive/'My Drive'/'Deep Learning Proj'/math.txt /content

# from path will differ depending on where you saved the zip file in Google Drive
! unzip -DD -q  /content/train.zip -d  /content/
! unzip -DD -q  /content/test.zip -d  /content/
! unzip -DD -q  /content/val.zip -d  /content/



Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
Length of datasets
'''
num_train_str = !ls train | wc -l
num_test_str = !ls test | wc -l
num_val_str = !ls val | wc -l
num_train = int(num_train_str[0])
num_test = int(num_test_str[0])
num_val = int(num_val_str[0])

print(f'Number of train images: {num_train}\nNumber of test images: {num_test}\nNumber of validation images: {num_val}\nTotal images: {num_train+num_test+num_val}')

Number of train images: 158480
Number of test images: 30637
Number of validation images: 6765
Total images: 195882


In [ ]:
train_root = "/content/train/"
test_root = "/content/test/"
val_root = "/content/val/"
label_file = "/content/math.txt"

In [ ]:
import os
import cv2
import torch.utils.data
from PIL import Image


class LatexDataset(torch.utils.data.Dataset):
  def __init__(self, transform=None, dataroot=train_root): # can change dataroot to be either train_root, test_root, val_root
        '''Initialize the dataset.'''
        self.transform = transform
        self.dataroot = dataroot
        self.labels_txt = label_file
        self._parse()

  def _parse(self):
        '''
        Parse the math.txt file.
        Populates the following private variables:
        - self.im_paths: A list of strings storing the associated image paths
        - self.labels: A list of strings, where each string is the latex code for an image
        '''
        def getImPath(idx):
            # Find image in either train, test, or validation folder
            imname = str(idx - 1).zfill(7) + '.png'
            if os.path.exists(f'{self.dataroot}{imname}'):
              impath = f'{self.dataroot}{imname}'
            else:
              return None

            try:
                Image.open(impath).verify()
            except Exception as e:
                # Some images can't be opened
                # print(f"Image at path {impath} is corrupted. Error: {e}")
                return None

            return impath

        self.im_paths = []
        self.labels = []

        with open(self.labels_txt) as f:
            for idx, line in enumerate(f):
                impath = getImPath(idx+1)

                if impath is not None:
                    labels = line.strip('\n')
                    if len(labels) < 700:
                      self.im_paths.append(impath)            # Image name
                      self.labels.append(labels)    # String of latex code


  def __len__(self):
        '''Return length of the dataset.'''
        assert len(self.labels) == len(self.im_paths)
        return len(self.labels)

  def __getitem__(self, index):
        '''
        Return the (image, attributes) tuple.
        This function gets called when you index the dataset.
        '''
        def img_load(index):
            imraw = Image.open(self.im_paths[index])
            imgray = imraw.convert('L')                         # Convert image to greyscale
            imthresh = imgray.point(lambda p: p > 240 and 255)  # Threshold image to remove background (white)
            im = self.transform(imthresh)
            return im

        target = self.labels[index]
        return img_load(index), target

In [ ]:
'''
Dictionary block: converts a LaTeX string to a dictionary of latex tokens, where
each unique token has its own entry and integer value assigned to it

'''
class LatexDict():
    def __init__(self, num_tokens=256):
        self.labels_txt = label_file
        self.num_tokens = num_tokens
        self.latex_dict = {'<UKN>':0, '<PAD>':1} # Initialize with token for unknown and for padding
        self.latex_dict_inverse = {0:'<UKN>', 1:'<PAD>'} # Initialize inverse dict for quicker reverse lookups
        self.create_dict()

    def create_dict(self):
        # Go through entire label file and populate dictionary
        with open(self.labels_txt) as f:
            for line in f:
                tokens = line.split()
                for token in tokens:
                    if token not in self.latex_dict:
                        # Assign a new ID for the unseen token
                        new_id = len(self.latex_dict)
                        self.latex_dict[token] = new_id
                        self.latex_dict_inverse[new_id] = token

    def map_tokens(self, tex_str_list, batch_size):
        ids_tensor = torch.full((batch_size, self.num_tokens), self.latex_dict['<PAD>'], dtype=torch.float32)

        for row, tex_str in enumerate(tex_str_list):
            tokens = tex_str.split()
            for col, token in enumerate(tokens):
                ids_tensor[row, col] = self.latex_dict[token]

        return ids_tensor

    def tokens_to_tex(self, token_vec):
        tex_str = ' '
        for token_id in token_vec.tolist():
            if token_id in self.latex_dict_inverse:
                if self.latex_dict_inverse[token_id] != '<PAD>' and self.latex_dict_inverse[token_id] != '<UKN>':
                    tex_str += self.latex_dict_inverse[token_id] + ' '

        return tex_str

    def __dict__(self):
        return self.latex_dict

    def __len__(self):
        return len(self.latex_dict)

# latex_dict = LatexDict()


Creating the CNN Block

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import math

class CNN_Block(nn.Module):
    def __init__(self):
        super(CNN_Block, self).__init__()
        # self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)    # Images are originally one channel, added padding as well
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)

        #self.fc1 = nn.Linear(256 * 2 * 8, 1024)  # Adjusted input size based on the output size of the convolutional layers
        #self.fc2 = nn.Linear(1024, 512)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        #print('x: shape:', x.shape)
        #x = x.view(64,-1)   # Flatten so this can be used in linear layers

        #x = F.relu(self.fc1(x))
        #x = self.fc2(x)

        return x


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        #self.num_layers = num_layers
        self.batch_size = batch_size

        self.cell_state = None
        self.hidden_state = None

        #Input-gate parameters
        self.W_i = nn.Parameter(torch.zeros(hidden_size, input_size + hidden_size))
        self.b_i = nn.Parameter(torch.zeros(hidden_size, 1))
        #forget_gate parameters
        self.w_f = nn.Parameter(torch.zeros(hidden_size,  input_size + hidden_size))
        self.b_f = nn.Parameter(torch.zeros(hidden_size, 1))
        #candidate parameters
        self.w_c = nn.Parameter(torch.zeros(hidden_size, input_size + hidden_size))
        self.b_c = nn.Parameter(torch.zeros(hidden_size, 1))
        #output gate parameters
        self.w_o = nn.Parameter(torch.zeros(hidden_size, input_size + hidden_size))
        self.b_o = nn.Parameter(torch.zeros(hidden_size, 1))

        self.init_weights()
        self.reset_LSTM_states(batch_size)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def reset_LSTM_states(self, batch_size):
        self.cell_state = torch.zeros(batch_size, self.hidden_size)
        self.hidden_state = torch.zeros(batch_size, self.hidden_size)

    def forward(self, x):
        #print('here',x.shape, self.hidden_state.shape)

        X_H = torch.cat((x.T.to(self.device), self.hidden_state.to(self.device)), 1)
        input_update = torch.sigmoid(torch.matmul(self.W_i.to(self.device), X_H.T.to(self.device)) + self.b_i.to(self.device))
        forget_update = torch.sigmoid(torch.matmul(self.w_f.to(self.device), X_H.T.to(self.device)) + self.b_f.to(self.device))
        #print('forget update',forget_update.shape)
        #print('input update',input_update.shape)
        candidate_update = torch.tanh(torch.matmul(self.w_c.to(self.device), X_H.T.to(self.device)) + self.b_c.to(self.device))
        #print('candi update',candidate_update.shape)
        #print('cell_state',self.cell_state.shape)
        self.cell_state = forget_update.T.to(self.device) * self.cell_state.to(self.device) + input_update.T.to(self.device) * candidate_update.T.to(self.device)
        output_update = torch.sigmoid(torch.matmul(self.w_o.to(self.device), X_H.T.to(self.device)) + self.b_o.to(self.device))
        self.hidden_state = output_update.T.to(self.device) * torch.tanh(self.cell_state.to(self.device))
        #print('hidden_state',self.hidden_state.shape)
        return self.hidden_state.T

In [ ]:
class attention(nn.Module):
    def __init__(self, beta_size, hidden_size, v_length):
        super(attention, self).__init__()
        #weights for the hidden layer
        self.w_h = nn.Linear(hidden_size, beta_size, bias=False)
        #weights for the encoded image
        self.w_v = nn.Linear(v_length, beta_size, bias=False)
        #weights for the betas
        self.w_beta = nn.Parameter(torch.Tensor(beta_size))
        nn.init.uniform_(self.w_beta, -1e-2, 1e-2)

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.init_weights()

    def init_weights(self):
        torch.nn.init.xavier_uniform_(self.w_h.weight)
        torch.nn.init.xavier_uniform_(self.w_v.weight)

    def forward(self, V_new, h_t):
        #Multiplication
        U_t = torch.tanh(self.w_h(h_t.to(self.device)).unsqueeze(1) + self.w_v(V_new.to(self.device))) # [B, H' * W', C]

        #activation + sum
        E_t = torch.sum(U_t.to(self.device) * self.w_beta.to(self.device), dim=-1)

        #activation
        A_t = torch.softmax(E_t.to(self.device), dim = 1).unsqueeze(1)

        C_t = torch.matmul(A_t.to(self.device), V_new.to(self.device)).squeeze(1)

        return C_t, A_t

In [ ]:
def PositionalEmbedding2D(D_model,height,width):
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  if D_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dimension (got dim={:d})".format(D_model))
  pe = torch.zeros(D_model,height,width)
  d_model = int(D_model / 2)
  div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
  pos_w = torch.arange(0., width).unsqueeze(1)
  pos_h = torch.arange(0., height).unsqueeze(1)
  pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
  pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
  pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
  pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)

  return pe


In [ ]:
class BeamSearch(nn.Module):

    def __init__(self, end_index: int,
                 max_steps: int = 50,
                 beam_size: int = 10,
                 per_node_beam_size: int = None) -> None:
        self._end_index = end_index
        self.max_steps = max_steps
        self.beam_size = beam_size
        self.per_node_beam_size = per_node_beam_size or beam_size

    def search(self, start_predictions, start_state, step):
        batch_size = start_predictions.size()[0]


In [ ]:
'''
Overall Model Class
'''

class Model(nn.Module):
    def __init__(self, embedding_size, hidden_size, batch_size, sequence_length, vocab_size, o_layer_size, v_length=256):
          super().__init__()
          # Static params
          self.v_length = v_length
          self.batch_size = batch_size
          self.sequence_length = sequence_length
          self.vocab_size = vocab_size
          self.o_layer_size = o_layer_size
          input_size = embedding_size + o_layer_size
          self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

          # Network Modules
          self.CNN = CNN_Block()
          self.LSTM_module = LSTM(input_size, hidden_size, batch_size)
          self.AttentionMechanism = attention(beta_size=256, hidden_size=hidden_size, v_length=v_length) # TODO: Change these hard-coded values

          # The other layers
          # self.E = nn.Parameter(torch.zeros(embedding_size, vocab_size)).double()
          self.E = nn.Embedding(vocab_size, embedding_size)
          self.O = nn.Linear(v_length + hidden_size, o_layer_size, bias=False)
          self.W_out = nn.Linear(o_layer_size, vocab_size, bias=False)
          self.softmax = nn.Softmax(1)

          # Initialization of h_t
          self.init_Wh = nn.Linear(v_length, hidden_size)

          # Beam search
          #self.beam_size = 3
          #self._beam_search = BeamSearch(self.vocab_size - 2, self.sequence_length, beam_size = self.beam_size)

    def forward(self, X_batch, labels):
        self.train()

        # 1) CNN, aka "HyperCube Creation" :)
        V = self.CNN(X_batch).to(self.device)

        # Transforming into a
        #print('V shape',V.shape)
        #print('V shape',V.shape)
        pe2 = (PositionalEmbedding2D(256, V.size(2), V.size(3)).repeat(self.batch_size,1,1,1)).to(self.device)
        #print(pe2.shape)
        V = V + pe2[:, :, : V.size(2), : V.size(3)]
        V = V.permute(0, 2, 3, 1)

        batch_size, H_prime, W_prime, C = V.shape
        V = torch.reshape(V, (batch_size, H_prime * W_prime, C))
        V = V
        # Pre-allocate memory
        output = torch.zeros(self.batch_size, self.sequence_length, self.vocab_size).to(self.device)

        # Initialize Y and O
        Y_t = (self.vocab_size - 3) * torch.ones(self.batch_size).to(self.device)
        O_t = torch.zeros(self.o_layer_size, self.batch_size).to(self.device)

        # Reset S_t
        self.LSTM_module.reset_LSTM_states(batch_size)

        # Initialize H_t
        mean_encoder_out = torch.mean(V, 1).to(self.device)
        #print('mean encoder out:',mean_encoder_out.shape)
        H_t = torch.transpose(torch.tanh(self.init_Wh(mean_encoder_out)), 0, 1).to(self.device)
        #print('H_t',H_t.shape)
        self.LSTM_module.H_t = H_t
        #print('y_t before',Y_t.shape)
        for i in range(self.sequence_length):
            #print('For loop : O_t , V', O_t.shape,V.shape)
            O_t, logits, _ = self.step_decoding(O_t.to(self.device), V.to(self.device), Y_t.to(self.device), True)

            output[:, i, :] = logits

            # Next in the sequence
            #Y_t = labels[:, i]

            Y_t = torch.argmax(logits, dim=1)
            #print('there, yt',Y_t.shape)

        #return output
        return output

    def step_decoding(self, O_t, V, Y_t, soft_max = True):
        # The input

        #print('hrererer')
        #print(Y_t)
        Embedded_vec = self.E(Y_t.long())
        X_t = torch.cat((torch.transpose(Embedded_vec, 0, 1), O_t), 0)
        #print(X_t.shape)
        # Update hidden states
        H_t = self.LSTM_module(X_t)

        # Attention mechanism
        C_t, _ = self.AttentionMechanism(V, torch.transpose(H_t, 0, 1))

        # O_t
        #print('ct',C_t.shape)
        #print('ht',H_t.shape)
        concat = torch.transpose(torch.cat((H_t, C_t.T), 0), 0, 1)
        #print('concat',concat.shape)
        linear_O = self.O(concat)
        O_t = torch.tanh(linear_O)
        logits = self.W_out(O_t)
        #print('logits',logits.shape)
        O_t = torch.transpose(O_t, 0, 1)
        #print('O-t',O_t.shape)

        if soft_max:
            logits = self.softmax(logits)
        return O_t, logits, (H_t, C_t)

    def Predict(self, logits):
      pass

    def forward_predict(self, X_batch):
        self.eval()
        # 1) CNN, aka "HyperCube Creation" :)
        with torch.no_grad():
            V = self.CNN(X_batch)

            # Transforming into a cube
            V = V.permute(0, 2, 3, 1)
            batch_size, H_prime, W_prime, C = V.shape
            V = torch.reshape(V, (batch_size, H_prime * W_prime, C))

            # Pre-allocate memory
            output = torch.zeros(batch_size, self.sequence_length, self.vocab_size).double()

            # Initialize Y and O
            Y_t = (self.vocab_size - 3) * torch.ones(batch_size).long()
            O_t = torch.zeros(self.o_layer_size, batch_size).double()

            # Reset S_t
            self.LSTM_module.reset_LSTM_states(batch_size)

            # Initialize H_t
            mean_encoder_out = torch.mean(V, 1)
            H_t = torch.transpose(torch.tanh(self.init_Wh(mean_encoder_out)), 0, 1)
            self.LSTM_module.H_t = H_t

            for i in range(self.sequence_length):
                O_t, logits, _ = self.step_decoding(O_t.double(), V.double(), Y_t.double(), True)

                output[:, i, :] = logits

                # Greedy approach
                Y_t = torch.argmax(logits, dim=1)

        return output

In [ ]:
'''
Initialize dataset and image preprocessing

NOTE:
    Some of the images in the dataset are corrupted. To deal with this,
    there is a check for each image to ensure that it can be loaded.
'''
import torchvision.transforms as transforms

reduced_imsize = (32, 128)  # Images are reduced to this size

# Define the transform pipeline - add normalization?
transform = transforms.Compose([
    transforms.Resize(reduced_imsize),
    transforms.ToTensor(),
])

train_dataset = LatexDataset(transform=transform, dataroot=train_root)
test_dataset = LatexDataset(transform=transform, dataroot=test_root)
val_dataset = LatexDataset(transform=transform, dataroot=val_root)

# Device settings
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda:0


In [ ]:
'''
Initialize hyperperameters, trainloader, and dictionary of LaTeX token mappings
'''

# Hyperparameters
batch_size = 32
learning_rate = 0.0001
weight_decay = 0.00001  # (L2 penalty)

max_tokens = 700        # Maximum number of tokens in a latex string
latex_dict = LatexDict(num_tokens=max_tokens)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8,drop_last=True)

# print(f'Dictionary length: {latex_dict.__len__()}')
# print(f'Dictionary: {latex_dict.__dict__()}')

In [ ]:
'''
Initialize Model

Initialize Loss Functions:
1. Normal Cross-Entropy Loss between prediction and label
2. LaTeX compile test:
    - Custom function, returns True if code can compile into LaTeX, False if not

Initialize Optimizer:
1. Adam Optimizer
'''

    # Hyperparameters
embedding_size = 80; # number of rows in the E-matrix
o_layer_size = 700  ;  # size of o-vektorn TODO: What should this be?
hidden_size = 700  ;
sequence_length = 700  ; vocab_size = 700  ;
batch_size = 32
n_epochs = 2
beam_size = 5
model = Model(embedding_size=embedding_size, hidden_size=hidden_size, batch_size=batch_size, sequence_length=sequence_length, vocab_size=vocab_size, o_layer_size = o_layer_size).to(device)
criterion = nn.CrossEntropyLoss()   ## May need to change this
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate, weight_decay=0)

In [ ]:
pbar_test = trainloader
images, y = next(iter(pbar_test))
y_vec = latex_dict.map_tokens(list(y), batch_size=batch_size)
print(y_vec)
latexx = latex_dict.tokens_to_tex(y_vec[0])
print(latexx)

print(y_vec.shape)


y_vec = torch.tensor(y_vec, dtype=torch.long)
y_vec = torch.LongTensor(y_vec)
one_hot = torch.nn.functional.one_hot(y_vec, num_classes=max_tokens)
print(one_hot.size())
print(one_hot)

In [ ]:
from tqdm import tqdm

'''
Training Loop
'''

num_epoch = 10

for epoch in range(num_epoch):
    print('epoch:', epoch)
    pbar = tqdm(trainloader)
    for images, y in pbar:

        images = images.to(device)              # Send to gpu

        y_vec = latex_dict.map_tokens(list(y), batch_size=batch_size)
        #print(y_vec)
        latexx = latex_dict.tokens_to_tex(y_vec[0])
        #print(latexx)
        y_vec = torch.tensor(y_vec, dtype=torch.long)
        y_vec = torch.LongTensor(y_vec)
        one_hot = torch.nn.functional.one_hot(y_vec, num_classes=max_tokens)
        one_hot = one_hot.to(device)                # Send to gpu
        #print(y_vec[0].shape)
        #print(images.shape)
        #V = images.reshape(images.shape[0], images.shape[1] * images.shape[2], images.shape[3])
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0)
        predictions = model.forward(images,one_hot)        # Get predictions

        print(torch.argmax(predictions[1]),torch.argmax(one_hot[1]))
        print(predictions.shape,one_hot.shape)
        one_hot = torch.tensor(one_hot, dtype=torch.float)
        #print('y_vec',y_vec.shape)
        #print('pred',predictions.shape)
        loss1 = criterion(predictions, y_vec.to(device))   # Calculate first loss function
        print(loss1)
        # loss2 = can_compile(precitions)         # Check if output can compile

        optimizer.zero_grad()

        loss1.backward()
        optimizer.step()


epoch: 0


  0%|          | 0/4921 [00:00<?, ?it/s]<ipython-input-34-e8c7956a7a1b>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_vec = torch.tensor(y_vec, dtype=torch.long)


tensor(1062, device='cuda:0') tensor(57, device='cuda:0')
torch.Size([32, 700, 700]) torch.Size([32, 700, 700])
tensor(6.5511, device='cuda:0', grad_fn=<NllLoss2DBackward0>)


<ipython-input-34-e8c7956a7a1b>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.tensor(one_hot, dtype=torch.float)
  0%|          | 1/4921 [00:02<3:51:02,  2.82s/it]

tensor(1062, device='cuda:0') tensor(53, device='cuda:0')
torch.Size([32, 700, 700]) torch.Size([32, 700, 700])
tensor(6.5511, device='cuda:0', grad_fn=<NllLoss2DBackward0>)


...

 46%|████▌     | 2245/4921 [1:17:25<1:31:49,  2.06s/it]

tensor(5607, device='cuda:0') tensor(176, device='cuda:0')
torch.Size([32, 700, 700]) torch.Size([32, 700, 700])
tensor(6.5497, device='cuda:0', grad_fn=<NllLoss2DBackward0>)


 46%|████▌     | 2246/4921 [1:17:27<1:31:34,  2.05s/it]

tensor(4907, device='cuda:0') tensor(136, device='cuda:0')
torch.Size([32, 700, 700]) torch.Size([32, 700, 700])
tensor(6.5497, device='cuda:0', grad_fn=<NllLoss2DBackward0>)


 46%|████▌     | 2247/4921 [1:17:29<1:32:02,  2.07s/it]